In [1]:
from ultralytics import YOLO
import torch
import pandas as pd
import torch
import os
import glob
import wandb
import ultralytics
import cv2
print('ultralytics %s' % ultralytics.__version__)

ultralytics 8.3.62


In [2]:
!nvidia-smi

Thu Feb 13 19:39:12 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.94                 Driver Version: 560.94         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1660 ...  WDDM  |   00000000:06:00.0  On |                  N/A |
| 32%   48C    P0             29W /  125W |     821MiB /   6144MiB |      1%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Arguments setting for the model

In [3]:
args = dict(
    data = "data.yaml",
    epochs = 5,
    patience = 5,
    batch = 4,
    save = False,
    save_period = 5,
    device = '0',
    workers = 1,
    project = "weights",
    name = "try",
    exist_ok = False, #overwrite the existing run for iterative testing, turn false when confident
    pretrained = True,
    optimizer = 'auto',
    resume = False,
    amp = False, #mixed precision training for faster training, might cause some issues
    fraction = 0.1, #fraction of dataset if resouce limited, change to 1 when confident
    profile = False, #enable ONNX and TensorRT profiling
    freeze = None, #freeze the first N layers or can specify the layers by index
    #for validation part
    conf = 0.01,
    iou = 0.6,
    degrees = 0.0, 
    shear = 0.0,
    perspective = 0.0,
    plots = True,
    half = True, #use half precision for weaker gpu
)

Loading the model yoloV8 and running wandb for logging

Training of model

In [4]:
#training model
model = YOLO('yolov8n.pt')


In [5]:
wandb.login()

wandb: Currently logged in as: ejento47 (object-detection-glottis-fyp) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [6]:
wandb.init(project='YOLOv8-demo')

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [7]:
results = model.train(**args)


New https://pypi.org/project/ultralytics/8.3.75 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.62  Python-3.10.8 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce GTX 1660 SUPER, 6144MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=5, time=None, patience=5, batch=4, imgsz=640, save=False, save_period=5, cache=False, device=0, workers=1, project=weights, name=try2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=False, fraction=0.1, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=0.01, iou=0.6, max_det=300, half=True, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False

train: Scanning D:\FYP Datasets\fyp\training\train\labels.cache... 4740 images, 296 backgrounds, 0 corrupt: 100%|██████████| 4740/4740 [00:00<?, ?it/s]
val: Scanning D:\FYP Datasets\fyp\training\val\labels.cache... 10665 images, 696 backgrounds, 0 corrupt: 100%|██████████| 10665/10665 [00:00<?, ?it/s]


Plotting labels to weights\try2\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 1 dataloader workers
Logging results to weights\try2
Starting training for 5 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/5      1.11G      1.455      2.359      1.199         10        640: 100%|██████████| 1185/1185 [02:24<00:00,  8.20it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1334/1334 [01:36<00:00, 13.78it/s]

                   all      10665       9969      0.815      0.738      0.799      0.542


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        5/5      1.11G      1.033     0.7502      1.016          7        640: 100%|██████████| 1185/1185 [01:51<00:00, 10.62it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1334/1334 [01:25<00:00, 15.61it/s]


                   all      10665       9969      0.876      0.854      0.878      0.664

5 epochs completed in 0.285 hours.
Optimizer stripped from weights\try2\weights\last.pt, 6.2MB
Optimizer stripped from weights\try2\weights\best.pt, 6.2MB

Validating weights\try2\weights\best.pt...
Ultralytics 8.3.62  Python-3.10.8 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce GTX 1660 SUPER, 6144MiB)
Model summary (fused): 168 layers, 3,006,818 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1334/1334 [01:18<00:00, 16.91it/s]


                   all      10665       9969      0.877      0.853      0.879      0.664
               Glottis       9969       9969      0.877      0.853      0.879      0.664
Speed: 0.3ms preprocess, 2.7ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to weights\try2


In [ ]:
print(results)
wandb.log()

wandb.finish()

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001F923BA7FA0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

NameError: name 'epoch' is not defined

Running Predictions on model using bagls video and dhs video

note we do not need to use openCV for video processing since we already have the video, this will not be true once we start using the webcamera

In [ ]:
dhs_video = r'D:\FYP Code\testfiles\dhs.mp4'
baglsC_video = r'D:\FYP Code\testfiles\bagls-color.mp4'
bagls_video = r'D:\FYP Code\testfiles\bagls.mp4'
test_path = r'D:\FYP Datasets\fyp\training\test\images'
weight_path = r'weights\try\weights\best.pt'

In [ ]:
model = YOLO(weight_path)
# model.predict(source= test_path, save_dir = 'results', save= True)
model.predict(source=dhs_video, save_dir = 'results', save= True)
model.predict(source=baglsC_video, save_dir = 'results', save= True)
model.predict(source=bagls_video, save_dir = 'results', save= True)


In [ ]:
# wandb.log({"final_mAP": results.metrics.mAP50})
# wandb.finish()